In [1]:
# ============================================================
# Parallel Cellular Algorithm (PCA)
# Application: Routing and Scheduling for Resource Allocation
# ============================================================

import numpy as np
import random
import multiprocessing
from copy import deepcopy

# ----------------------------
# Step 1: Define the problem
# ----------------------------
NUM_TASKS = 10        # Number of tasks to schedule
NUM_RESOURCES = 3     # Available resources
TASK_TIMES = np.random.randint(5, 15, size=NUM_TASKS)  # random processing times

# Fitness function: minimize total completion time (makespan)
def fitness(solution):
    """
    solution: array of length NUM_TASKS, where each element = assigned resource
    returns: total completion time (makespan)
    """
    completion = np.zeros(NUM_RESOURCES)
    for task_id, resource_id in enumerate(solution):
        completion[resource_id] += TASK_TIMES[task_id]
    return completion.max()  # The larger the makespan, the worse the fitness


# ---------------------------------
# Step 2: Define genetic operators
# ---------------------------------
def crossover(parent1, parent2):
    """Single-point crossover"""
    point = random.randint(1, NUM_TASKS - 2)
    child = np.concatenate((parent1[:point], parent2[point:]))
    return child

def mutate(solution, p_m=0.1):
    """Randomly assign a task to a different resource"""
    new_sol = solution.copy()
    for i in range(NUM_TASKS):
        if random.random() < p_m:
            new_sol[i] = random.randint(0, NUM_RESOURCES - 1)
    return new_sol


# ---------------------------------
# Step 3: Initialize cellular grid
# ---------------------------------
GRID_SIZE = (5, 5)  # 5x5 grid
population = np.array([[np.random.randint(0, NUM_RESOURCES, NUM_TASKS)
                        for _ in range(GRID_SIZE[1])]
                        for _ in range(GRID_SIZE[0])])

# Neighborhood function (Moore neighborhood)
def get_neighbors(i, j):
    neighbors = []
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            if di == 0 and dj == 0:
                continue
            ni, nj = (i + di) % GRID_SIZE[0], (j + dj) % GRID_SIZE[1]
            neighbors.append((ni, nj))
    return neighbors


# ---------------------------------
# Step 4: PCA evolution loop
# ---------------------------------
MAX_GENERATIONS = 100
p_c = 0.8   # crossover probability
p_m = 0.1   # mutation probability

best_solution = None
best_fitness = float('inf')

for gen in range(MAX_GENERATIONS):
    new_population = deepcopy(population)

    # Parallel update using multiprocessing
    def evolve_cell(i, j):
        current = population[i][j]
        # Select neighbors
        neighbors = get_neighbors(i, j)
        # Pick a random neighbor to mate
        ni, nj = random.choice(neighbors)
        partner = population[ni][nj]

        # Evaluate
        fit_cur = fitness(current)
        fit_partner = fitness(partner)

        # Selection: better one becomes parent1
        if fit_partner < fit_cur:
            parent1, parent2 = partner, current
        else:
            parent1, parent2 = current, partner

        # Crossover
        if random.random() < p_c:
            child = crossover(parent1, parent2)
        else:
            child = parent1.copy()

        # Mutation
        child = mutate(child, p_m)

        # Replacement
        fit_child = fitness(child)
        if fit_child < fit_cur:
            new_population[i][j] = child
        else:
            new_population[i][j] = current

        return i, j, new_population[i][j], fit_child

    # Run in parallel for all cells
    pool = multiprocessing.Pool()
    results = pool.starmap(evolve_cell, [(i, j) for i in range(GRID_SIZE[0]) for j in range(GRID_SIZE[1])])
    pool.close()
    pool.join()

    # Update population and track best
    for i, j, sol, fit_val in results:
        population[i][j] = sol
        if fit_val < best_fitness:
            best_fitness = fit_val
            best_solution = sol.copy()

    # Print progress
    if gen % 10 == 0 or gen == MAX_GENERATIONS - 1:
        print(f"Generation {gen} - Best Fitness: {best_fitness:.2f}")

# ---------------------------------
# Step 5: Display final solution
# ---------------------------------
print("\nBest Routing & Scheduling Solution Found:")
print("Task -> Resource assignment:", best_solution)
print("Task processing times:", TASK_TIMES)
print("Final makespan (total completion time):", best_fitness)


Generation 0 - Best Fitness: 38.00
Generation 10 - Best Fitness: 37.00
Generation 20 - Best Fitness: 37.00
Generation 30 - Best Fitness: 37.00
Generation 40 - Best Fitness: 37.00
Generation 50 - Best Fitness: 37.00
Generation 60 - Best Fitness: 37.00
Generation 70 - Best Fitness: 37.00
Generation 80 - Best Fitness: 37.00
Generation 90 - Best Fitness: 37.00
Generation 99 - Best Fitness: 37.00

Best Routing & Scheduling Solution Found:
Task -> Resource assignment: [0 1 0 2 1 2 1 0 2 1]
Task processing times: [ 5  8 13 13 13 12  8 14 12  8]
Final makespan (total completion time): 37.0
